# Saving and loading populations

Another feature Brush implements is the ability to save and load entire populations.
We use JSON notation to store the population into a file that is human readable. The same way, we can feed an estimator a previous population file to serve as starting point for the evolution.

In this notebook, we will walk through how to use the `save_population` and `load_population` parameters. 

We start by getting a sample dataset and splitting it into `X` and `y`:

In [ ]:
import pandas as pd
from pybrush import BrushRegressor

# load data
df = pd.read_csv('../examples/datasets/d_enc.csv')
X = df.drop(columns='label')
y = df['label']

To save the population after finishing the evolution, you nee to set `save_population` parameter to a value different than an empty string. Then, the final population is going to be stored in that specific file.

In this example, we create a temporary file.

In [ ]:
import pickle
import os, tempfile

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

# set verbosity==2 to see the full report
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    save_population=pop_file,
    verbosity=2
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Loading a previous population is done providing `load_population` a string value corresponding to a JSON file generated by Brush. In our case, we will use the same file from the previous code block.

After loading the population, we run the evolution for 10 more generations, and we can see that the first generation started from the previous population. This means that the population was successfully saved and loaded.

In [ ]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

You can open the serialized file and change individuals' programs manually.

This also allow us to have checkpoints in the execution.

## Using population files with classification

To give another example, we do a two-step fit in the cells below.

First, we run the evolution and save the population to a file; then, we load it and keep evolving the individuals.

What is different though is that the first run is optimizing `error` and `complexity`, and the second run is optimizing `average_precision_score` and `size`.

In [ ]:
from pybrush import BrushClassifier

# load data
df = pd.read_csv('../examples/datasets/d_analcatdata_aids.csv')
X = df.drop(columns='target')
y = df['target']

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    scorer="log",
    save_population=pop_file,
    verbosity=2
)

est.fit(X,y)
print(est.best_estimator_.get_model())

y_pred = est.predict(X)
print('score:', est.score(X,y))

In [ ]:
est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    #load_population=pop_file,
    objectives=["error", "complexity"],
    scorer="average_precision_score",
    max_gens=1,
    verbosity=2
)

est.fit(X,y)
print(est.best_estimator_.get_model())

y_pred = est.predict(X)
print('score:', est.score(X,y))

In [ ]:
est.best_estimator_.get_model()